In [15]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

In [16]:
root = '/modelo_recomendacion/'
df1=pd.read_csv(root + 'tmdb_5000_credits.csv')
df2=pd.read_csv(root + 'tmdb_5000_movies.csv')

df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

In [17]:
len(df2)

4803

In [18]:
df2 = df2[df2['original_title'].str.contains(r'^[a-zA-Z0-9 ]*$', na=False)]

In [19]:
len(df2)

3880

In [20]:
# Parseamos esas listas de diccionarios con un bucle y una funcion de la libreria ast
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [21]:
# Usamos esta función para extraer el director
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [22]:
# Traemos una lista con los tres primeros elementos
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #verificamos si hay mas de tres elementos, si los hay traemos solo tres.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return lista vacia si no hay datos
    return []

In [23]:
# Aplicamos las funciones
df2['director'] = df2['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [24]:
# Vemos como han quedado nuestras variables
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan,"[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"


In [25]:
# El siguiente paso sería convertir los nombres y las instancias de palabras clave en minúsculas y eliminar todos los espacios entre ellos. Esto se hace para que nuestro vectorizador no cuente al Johnny de "Johnny Depp" y al "Johnny Galecki" como iguales.
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:

        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [26]:
# Aplicamos la función
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [27]:
# Ahora estamos en condiciones de crear nuestra "sopa de metadatos", que es una cadena que contiene todos los metadatos que queremos enviar a nuestro vectorizador (a saber, actores, director y palabras clave).
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)

In [28]:
# Vemos como han quedado nuestras variables
df2[['title', 'cast', 'director', 'keywords', 'genres', 'soup']].head(3)

,title,cast,director,keywords,genres,soup
0,Avatar,"[samworthington, zoesaldana, sigourneyweaver]",jamescameron,"[cultureclash, future, spacewar]","[action, adventure, fantasy]",cultureclash future spacewar samworthington zo...
2,Spectre,"[danielcraig, christophwaltz, léaseydoux]",sammendes,"[spy, basedonnovel, secretagent]","[action, adventure, crime]",spy basedonnovel secretagent danielcraig chris...
3,The Dark Knight Rises,"[christianbale, michaelcaine, garyoldman]",christophernolan,"[dccomics, crimefighter, terrorist]","[action, crime, drama]",dccomics crimefighter terrorist christianbale ...


In [29]:
# Los siguientes pasos son los mismos que hicimos con nuestro recomendador basado en la descripción de la trama. Una diferencia importante es que usamos **CountVectorizer()** en lugar de TF-IDF. Esto se debe a que no queremos restar importancia a la presencia de un actor/director si ha actuado o dirigido en relativamente más películas.
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [30]:
# Computamos la similaridad del coseno en nuestro vector de palabras
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(count_matrix, count_matrix)

In [32]:
# Invertimos el index y el titulo de las peliculas otra vez
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [33]:
def get_recommendations(title, cosine_sim=similarity_matrix):
    # Obtiene el indice que coincide con el título
    idx = indices[title]

    # Conseguimos los pares de similaridades entre todas las peliculas con esa pelicula
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenamos las películas segun su puntaje de similaridad
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Hacemos un top 10
    sim_scores = sim_scores[1:11]

    # Obtenemos los índices de esas películas
    movie_indices = [i[0] for i in sim_scores]

    # Devolvemos el listado
    return df2['title'].iloc[movie_indices]

In [34]:
get_recommendations('The Dark Knight Rises')

40        The Dark Knight
82          Batman Begins
929          The Prestige
2483    Romeo Is Bleeding
2688       Black November
1193               Takers
1593               Faster
215              Catwoman
570        Gangster Squad
980         Kiss of Death
Name: title, dtype: object

In [35]:
np.save('similarity_matrix.npy', similarity_matrix)
indices.to_pickle('indices.pkl')
#df2['title'].to_pickle('df2.pkl')